In [2]:
:dep uuid = {features = ["v4"]}

In [3]:
use std::collections::HashMap;
use std::option::Option;
use uuid::Uuid;

use std::fmt;

# Устройства

In [4]:
#[derive(Debug, Clone)]
struct SmartOutlet {
    description: String,
    power: u8
}

#[derive(Debug, Clone)]
struct SmartThermometer {
    description: String,
    temperature: u8
}

In [5]:
impl fmt::Display for SmartOutlet {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        write!(f, "(Description: {}\nPower: {})", self.description, self.power)
    }
}

impl fmt::Display for SmartThermometer {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        write!(f, "(Description: {}\nTemperature: {})", self.description, self.temperature)
    }
}

In [6]:
impl SmartOutlet{
    fn new(description: String, power: Option<u8>) -> Self {
        SmartOutlet {
            description: description,
            power: power.unwrap_or(0)
        }
    }
}

In [7]:
impl SmartThermometer{
    fn new(description: String, temperature: Option<u8>) -> Self {
        SmartThermometer {
            description: description,
            temperature: temperature.unwrap_or(0)
        }
    }
}

# Общая имплементация устройств

In [8]:
#[derive(Debug, Clone)]
struct SmartDevice<I :fmt::Display> {
    id: Uuid, // у каждого девайса должен быть уникальный номер
    name: String, // у каждого девайса должено быть имя
    on: bool, // каждый девайс может быть или работать или нет
    config: I // каждый девайс имеет свой тип информации о себе, который можно прочитать
}

In [9]:
impl<I :fmt::Display> fmt::Display for SmartDevice<I> {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        write!(f, "(Name: {}\nOn: {}\n{})", self.name, self.on, self.config)
    }
}

In [10]:
impl<I:fmt::Display> SmartDevice<I>{
    fn new(name: String, config: I, on: Option<bool>) -> Self {
        SmartDevice {
            id: Uuid::new_v4(),
            name: name,
            on: on.unwrap_or(false),
            config: config
        }
    }
}

In [11]:
impl<I:fmt::Display> SmartDevice<I>{
    pub fn get_name(device:&Self) -> String {
        device.name.clone()
    }
}

# Список типов устройств

In [12]:
#[derive(Debug, Clone)]
enum Device {
    Outlet(SmartDevice<SmartOutlet>),
    Thermometer(SmartDevice<SmartThermometer>)
}

In [25]:
macro_rules! get_device {
    ($ex_af_device:expr) => {
        match &$ex_af_device {
            Device::Outlet(device) => device,
            Device::Thermometer(device) => device
        }
    };
}

In [27]:
macro_rules! fn_device {
    ($ex_af_device:expr, $op:expr) => {
        match &$ex_af_device {
            Device::Outlet(device) => $op(device),
            Device::Thermometer(device) => $op(device)
        }
    };
}

In [28]:
impl fmt::Display for Device {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            Device::Outlet(device) => write!(f, "{}", device),
            Device::Thermometer(device) => write!(f, "{}", device)
        }
    }
}

# Комната

In [15]:
#[derive(Clone)]
struct SmartRoom{
    name: String,
    devices: HashMap<String, Device>
}

In [16]:
#[derive(Debug)]
struct RoomError;

impl fmt::Display for RoomError {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> fmt::Result {
        write!(f, "Room not found")
    }
}

impl std::error::Error for RoomError {}

In [17]:
impl SmartRoom {
    fn get(&self, devices: Option<Vec<String>>) -> Vec<&Device> {
        if devices.is_some(){
            self.devices
                .iter()
                .filter_map(|(k, v)| if devices.clone().unwrap().contains(k) { Some(v) } else { None })
                .collect::<Vec<_>>().clone()
        } else {
            self.devices.values().collect::<Vec<_>>().clone()
        }
    }
    
    fn report(&self, devices: Option<Vec<String>>) -> fmt::Result {
        println!("Room Name: {}", self.name);
        println!("Devices:");
        for device in self.get(devices){
            println!("{}", device)
        }
        Ok(())
    } 
}

In [18]:
impl SmartRoom {
    fn get_devices_name(&self) -> Vec<&String> {
        Vec::from_iter(
            self.devices.keys()
        )
    }
    
    fn add_devices(&mut self, device: Device) -> fmt::Result{
        self.devices.insert(
            fn_device!(device, SmartDevice::get_name),
            device
        );
        Ok(())
    }
    
}

# Дом

In [19]:
#[derive(Clone)]
struct SmartHouse{
    id: Uuid,
    name: String,
    rooms: HashMap<String, SmartRoom>
}

In [20]:
impl SmartHouse {
    fn new(name: String) -> Self {
        // инициализация дома
        SmartHouse {
            id: Uuid::new_v4(),
            name: name,
            rooms: HashMap::new()
        }
    }

    fn add_room(&mut self, name: String) -> fmt::Result{
        self.rooms.insert(
            name.clone(),
            SmartRoom {
                name: name,
                devices: HashMap::new()
            }
        );
        Ok(())
    }

    fn get_rooms(&self) -> Vec<&String> {
        // список комнат
        Vec::from_iter(self.rooms.keys())
    }
}

In [21]:
impl SmartHouse {
    fn add_device(&mut self, room: String, device: Device) -> Result<(), RoomError>{
        let smartroom = self.rooms
            .get_mut(&room)
            .ok_or(RoomError {})?;
        //SmartRoom::add_devices(smartroom, device);
        smartroom.add_devices(device);
        Ok( () )
    }
}

In [22]:
impl SmartHouse {
    fn get(&self, rooms: Option<Vec<String>>) -> Vec<&SmartRoom> {
        if rooms.is_some(){
            self.rooms
                .iter()
                .filter_map(|(k, v)| if rooms.clone().unwrap().contains(k) { Some(v) } else { None })
                .collect::<Vec<_>>().clone()
        } else {
            self.rooms.values().collect::<Vec<_>>().clone()
        }
    }
}

In [23]:
impl SmartHouse {
    fn devices(&self, room: String) -> Result<Vec<&String>, RoomError> {
        // "список устройств в комнате `room`"
        let smartroom = self.rooms
            .get(&room)
            .ok_or(RoomError {})?;
        Ok( smartroom.get_devices_name() )
    }

    fn report(&self, rooms: Option<Vec<String>>) -> fmt::Result{
        for room in self.get(rooms){
            room.report(None);
        }
        Ok(())
    } 
}

In [24]:
// Инициализация устройств

In [25]:
let socket1 = SmartOutlet {
    description: "Розетка 1".to_string(),
    power: 220
};

In [26]:
let socket2 = SmartOutlet {
    description: "Розетка 2".to_string(),
    power: 220
};

In [27]:
let thermo = SmartThermometer {
    description: "Термометр 1".to_string(),
    temperature: 25
};

In [28]:
// Инициализация дома

In [29]:
let mut house = SmartHouse::new("Дом 1".to_string());

In [30]:
house.report(None)

Ok(())

In [31]:
house.get_rooms()

[]

In [32]:
house.devices("Комната 1".to_string())

Err(RoomError)

In [33]:
house.add_room("Комната 1".to_string())

Ok(())

In [34]:
house.report(None)

Room Name: Комната 1
Devices:


Ok(())

In [35]:
house.get_rooms()

["Комната 1"]

In [36]:
house.devices("Комната 1".to_string())

Ok([])

In [37]:
let room_vec = vec!["Комната 1".to_string()];

In [38]:
house.report(Some(room_vec))

Room Name: Комната 1
Devices:


Ok(())

In [47]:
let socket1_device = Device::Outlet(
    SmartDevice::new(
        "Розетка-1".to_string(),
        socket1.clone(),
        None
    )
);

In [48]:
house.add_device(
    "Комната 1".to_string(),
    socket1_device.clone()
)

Ok(())

In [49]:
house.devices("Комната 1".to_string())

Ok(["Розетка-1"])

In [50]:
house.report(None)

Room Name: Комната 1
Devices:
(Name: Розетка-1
On: false
(Description: Розетка 1
Power: 220))


Ok(())

In [51]:
let room_vec = vec!["Комната 1".to_string()];

house.report(Some(room_vec))

Room Name: Комната 1
Devices:
(Name: Розетка-1
On: false
(Description: Розетка 1
Power: 220))


Ok(())

In [39]:
let socket1 = SmartOutlet {
    description: "Розетка 1".to_string(),
    power: 220
};

let d:Device = Device::Outlet(
    SmartDevice::new(
        "test".to_string(),
        socket1,
        None
    )
);

let result = get!(
                    d,
                    SmartDevice::get_config
                );

Error: `match` arms have incompatible types